# PART 3/3: SIMULATING SPECIFIC MOLECULES
# MMM 2024 - 20.3.2024
## Daniele Passerone


The molecules you will explore today are:

1. C2H2 (acetylene)
2. CH4 (methane)
3. O2 (triplet oxygen)
4. hexatriene (C6H8)
5. Benzene (C6H6)

## Prerequisites

We assume that you already have learned how to run a geo opt workchain as well as an STM/Orbitals run. 
You will have to run them for the molecules listed above, for each of them changing slightly the parameters (for example, number of filled orbitals).
For the analysis of the simulations, you will need to keep track of the pk of the simulation, pk of the trajectory and so on, as explained in the notebook 2/3.


In [ ]:
#
# some important Imports...
#
import numpy as np
from ase import Atoms
from ase.io import read
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv

#
# and definitions of visualization functions (see last exercises)
#
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

## 0. Example with the acetylene molecule

The following assumes that you have run the **geo opt** of C2H2 as well as the **SCANNING PROBE MICROSCOPY/ORBITALS** run. The other molecule can be faced in a similar way.

## 1. Visualizing the optimization trajectory 
As shown in the **Notebook 2** you need the pk of the optimization trajectory, that you will insert in the next cell as value of **trajpk**.

In [ ]:

%load_ext aiida
%aiida

trajpk = 129
traj = load_node(trajpk)
symbols = traj.symbols
trajase=[traj.get_step_structure(step-1).get_ase() for step in traj.get_stepids()]
for a in trajase:
    a.set_pbc([False,False,False])

In [ ]:
view_trajectory(trajase)

#
# The optimization trajectory is shown.
#

## 2. Visualizing the orbitals
Now we compute the orbitals. To this end we have already finished (see **Notebook 2**) the simulation of the orbitals, and hit the "Cube creation kit" button. Keep the **pk** of the SPM calculation ready.

In the Exercise_5 directory, there is a script "run_cube_from_wfn_acetylene.sh" that is able to take some files from the cp2k SPM simulation (wavefunction) and transform it into the orbital cube files. The important things in this file is the number of occupied and unoccupied orbitals, that should correspond to the ones that you have indicated when launching the SPM AiiDAlab workchain. In the case of acetylene, 10 electrons, 5 occupied orbitals.

In [ ]:
!cat run_cube_from_wfn_acetylene.sh

## Generating the cubefiles, using the "cube-kit" set of files and the above script. 
Now we are ready to generate the orbital cube files. We replace my_pk below **with the pk of the SPM workchain**

In [ ]:
#
# Creating the cube file of the orbitals
# 
my_pk = 133
!rm -Rf ./cube-kit-pk{my_pk}*
!cp /home/jovyan/apps/surfaces/tmp/cube-kit-pk{my_pk}.zip .
!unzip cube-kit-pk{my_pk}.zip
!cp run_cube_from_wfn_acetylene.sh ./cube-kit-pk{my_pk}
!cd ./cube-kit-pk{my_pk} ; bash run_cube_from_wfn_acetylene.sh 

print ("*****************************\n\nTHE GENERATED FILES ARE:\n")
!ls ./cube-kit-pk{my_pk}/cubes/*cube
print ("\n*****************************\n")

## Visualizing HOMO and LUMO separately


We note that the name of the HOMO file above, we copy it into "file" and we read the cube file and its energy.

In [ ]:
file = './cube-kit-pk133/cubes/S0_5_HOMO.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:13])
view_homo=nv.NGLWidget()
caption_homo = "HOMO: E= "+ene+" eV"
view_homo.add_component(nv.ASEStructure(atoms))
c_2 = view_homo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_homo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#

print (caption_homo)
view_homo

We do the same for the LUMO, we simply need to change the name of the file.

In [ ]:
file = './cube-kit-pk133/cubes/S0_6_LUMO.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:13])
view_lumo=nv.NGLWidget()
caption_lumo = "LUMO: E= "+ene+" eV"
view_lumo.add_component(nv.ASEStructure(atoms))
c_2 = view_lumo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_lumo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#
print (caption_lumo)
view_lumo

We now create a combined view that visualizes orbital and energy:

In [ ]:
import ipywidgets as widgets
widg_caption_homo = widgets.Text(caption_homo)
combined_w_homo=widgets.HBox([view_homo,widg_caption_homo])

widg_caption_lumo = widgets.Text(caption_lumo)
combined_w_lumo=widgets.HBox([view_lumo,widg_caption_lumo])


In [ ]:
combined_w_homo

In [ ]:
combined_w_lumo

## Visualizing all orbitals together

We will use a loop and arrays to caption all orbitals and plot a matrix of representations. 

In [ ]:
#
# this is the part to create the visualizations and captions. Adapt nhomo and nlumo!!!
# 

nhomo = 5
nlumo = 5

nfile = 1
nhomonow = nhomo
nlumonow = -1
mydict = {}

views =[]
captions = []
filenames = []

for i in range (nhomo+nlumo):
    if (nfile <= nhomo):
        nhomonow = nhomonow-1
        midfix = 'HOMO'
        ind = nhomonow
        strind = '-'+str(ind)
    else:
        nlumonow = nlumonow+1
        midfix = 'LUMO'
        ind = nlumonow
        strind = '+'+str(ind)
    if (ind == 0):
        strind = ''
    totstring = "S0_"+str(nfile)+'_'+midfix+strind+'.cube'
    nfile = nfile+1
    myfile = './cube-kit-pk' + str(my_pk) + '/cubes/' + totstring
    atoms = read(myfile)
    filenames.append(myfile)
    print (nfile-2,myfile)
    file = myfile
    a=!head -2 {file} | tail -1
    b = str(a)
    ene=(b[4:13])
    views.append(nv.NGLWidget())
    captions.append(midfix+strind+" E= "+ene+" eV")
    views[nfile-2].add_component(nv.ASEStructure(atoms))
    c_2 = views[nfile-2].add_component(file)
    c_2.clear()
    c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
    c_3 = views[nfile-2].add_component(file)
    c_3.clear()
    c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)

In [ ]:
#
# And finally the visualization itself
#

import ipywidgets as widgets

myarray = []
for a in range(nhomo+nlumo):
    myarray.append(views[a])
    

caption =[]

for l in captions:
    caption.append(widgets.Text(l))

combined_w2 = []
for i in range(len(caption)):
    combined_w2.append(widgets.HBox([myarray[i],caption[i]]))


combined_widgets = widgets.VBox(combined_w2)
combined_widgets

## Including the MATRIX VISUALIZATION OF ALL ORBITALS into a function

The function allows to choose the isosurface, and the first orbital to visualize in the array, as well as the last one. Note that for larger molecules (benzene) you better choose a few orbitals at the time, not to "kill" your jupyter.



In [ ]:
def show_orbitals(molname,pk,nhomo,nlumo,isosurf=0.02,nfirstview=0,nlastview=5000):
    #
    # Click "cube creation kit" on the Orbitals aiidalab page ; note the pk
    #
    my_pk = pk
    !rm -Rf ./cube-kit-pk{my_pk}* ;
    !cp /home/jovyan/apps/surfaces/tmp/cube-kit-pk{my_pk}.zip . ;
    string = 'run_cube_from_wfn_'+molname+'.sh'
    !unzip cube-kit-pk{my_pk}.zip ;
    !cp {string} ./cube-kit-pk{my_pk} ;
    !cd ./cube-kit-pk{my_pk} ; bash {string} ; ls cubes/*cube ;
    #
    # Create the visualization of the orbitals
    #
    nfile = 1
    nhomonow = nhomo
    nlumonow = -1
    mydict = {}

    views =[]
    captions = []
    filenames = []

    for i in range (nhomo+nlumo):
        if (nfile <= nhomo):
            nhomonow = nhomonow-1
            midfix = 'HOMO'
            ind = nhomonow
            strind = '-'+str(ind)
        else:
            nlumonow = nlumonow+1
            midfix = 'LUMO'
            ind = nlumonow
            strind = '+'+str(ind)
        if (ind == 0):
            strind = ''
        totstring = "S0_"+str(nfile)+'_'+midfix+strind+'.cube'
        nfile = nfile+1
        myfile = './cube-kit-pk' + str(my_pk) + '/cubes/' + totstring
        atoms = read(myfile)
        filenames.append(myfile)
#        print (nfile-2,myfile)
        file = myfile
        a=!head -2 {file} | tail -1
        b = str(a)
        ene=(b[4:13])
        views.append(nv.NGLWidget())
        captions.append(midfix+strind+" E= "+ene+" eV")
        views[nfile-2].add_component(nv.ASEStructure(atoms))
        c_2 = views[nfile-2].add_component(file)
        c_2.clear()
        c_2.add_surface(color='blue', isolevelType="value", isolevel=-isosurf, opacity=0.5)
        c_3 = views[nfile-2].add_component(file)
        c_3.clear()
        c_3.add_surface(color='red', isolevelType="value", isolevel=isosurf, opacity=0.5)
    #
    # Visualize the orbitals and energy
    #
    import ipywidgets as widgets
    myarray = []
    for a in range(nhomo+nlumo):
        myarray.append(views[a])
    caption =[]
    for l in captions:
        caption.append(widgets.Text(l))
    combined_w2 = []
    for i in range(len(caption)):
        combined_w2.append(widgets.HBox([myarray[i],caption[i]]))
    combined_widgets = widgets.VBox(combined_w2[nfirstview:nlastview])
    return combined_widgets

In [ ]:
#
# In this way, the molecule can be visualized with a call to the function followed by a call of the molecule itself
#
acetylene = show_orbitals('acetylene',pk=133,nhomo=5,nlumo=5,nfirstview=0,nlastview=2,isosurf=0.01);


In [ ]:
#
# Visualizing the orbitals of acetylene:
#
acetylene

In [ ]:
#

## Now you are ready to generalize the exercise to the following molecules:

- H2
- CH4 (Methane)
- O2 (triplet state)
- Hexatriene
- Benzene 

### Note: for O2, you need a SPIN POLARIZED calculation (check the UKS option) with multiplicity 3 (also in the Orbital calculation)


You will get two sets of cube files, work only with the "S0" ones (spin up). Also, use n_homo 7 and n_lumo 5

### For Hexatriene vs. Benzene see the following [link](https://www.masterorganicchemistry.com/2017/05/05/the-pi-molecular-orbitals-of-benzene/)



# Assignments

1. For each molecule, draw a molecular orbital table filling the orbitals up to the correct level.
2. Discuss the difference between the H2 and O2 molecule.
3. Discuss the differences between the CH4 and CH2  and C6H6  molecules (hybridisation?)
4. Follow the discussion that you find in the link, and compare with your result. Discuss the differences you find between Hexatriene and Benzene
5. Which molecule has the largest Band Gap?
6. Apply a deformation to benzene and optimize again. Show the trajectory of the optimization. 